In [1]:
# https://towardsdatascience.com/text-summarization-using-deep-neural-networks-e7ee7521d804/  
https://saadsohail5104.medium.com/understanding-padding-in-nlp-types-and-when-to-use-them-bacae6cae401
https://github.com/rohithreddy024/Text-Summarizer-Pytorch/blob/master/data_util/data.py
https://www.ijcaonline.org/archives/volume185/number15/patil-2023-ijca-922837.pdf
https://www.kaggle.com/code/getanmolgupta01/text-summarization-lstm-encoder-decoder
https://github.com/rohithreddy024/Text-Summarizer-Pytorch/blob/master/model.py
https://arxiv.org/abs/1508.04025
https://www.freecodecamp.org/news/what-is-rouge-and-how-it-works-for-evaluation-of-summaries-e059fb8ac840/#:~:text=ROUGE%20stands%20for%20Recall%2DOriented,as%20well%20as%20machine%20translations.&text=If%20we%20consider%20just%20the,and%20reference%20summary%20is%206.
https://www.kaggle.com/code/quadeer15sh/introduction-to-language-modelling-using-rnns
https://www.kaggle.com/code/divyanshvishwkarma/seq2seq-model-with-attention-teacher-forcing-tf # nguồn data

SyntaxError: invalid decimal literal (2005343661.py, line 4)

In [ ]:
Multi-document summarization: Nghiên cứu từ PMC (2020) so sánh BART, T5, và PEGASUS trong bối cảnh zero-shot và few-shot cho tóm tắt đa tài liệu. Họ nhận thấy rằng với dữ liệu fine-tuning hạn chế (10 mẫu), không có sự khác biệt đáng kể về chất lượng tóm tắt giữa các mô hình, cho thấy ensemble có thể không mang lại lợi ích lớn trong trường hợp này.

In [ ]:
weight decay chỉ áp dụng cho weight, exclucde bias & LayerNorm

In [ ]:
The tokenizer is designed specifically for the Pegasus model, using a sentencepiece-based subword tokenization (BPE-like) that aligns well with the model’s input expectations. This ensures that tokenized inputs are formatted correctly for the Pegasus encoder-decoder architecture.

In [ ]:
Chúng ta sẽ sử dụng phương pháp Weighted Voting Ensemble kết hợp với Post-processing. Phương pháp này tận dụng điểm mạnh của từng mô hình (BART mạnh về ngữ pháp và tính mạch lạc, T5 tốt về độ ngắn gọn, Pegasus tốt về việc nắm bắt ý chính) bằng cách kết hợp các tóm tắt được tạo ra từ mỗi mô hình dựa trên trọng số (weight) được xác định từ hiệu suất trên tập validation (ví dụ: ROUGE hoặc BERTScore). Sau đó, một bước hậu xử lý sẽ được áp dụng để đảm bảo tóm tắt cuối cùng mạch lạc và không trùng lặp.